# Multiscale MILP with Materials and Emission

This is a continuation of 'One Location, Multiple Temporal Scales, Multiple Operations, Mixed Integer Linear Programming Example' [Example 2]. 

We will be adding the following considerations: 

1. material for the establishment of operations 
2. global warming potential induced as a sum of our decisions and choices 

## See Example 2

In [18]:
from energia import *

m = Model('example3')
m.q = Periods()
m.y = 4 * m.q


In [19]:
m.usd = Currency()
m.declare(Resource, ['power', 'wind', 'solar'])
m.solar.consume(m.q) <= 100
m.wind.consume <= 400
m.power.release.prep(180) >= [0.6, 0.7, 0.8, 0.3]

m.wf = Process()
m.wf(m.power) == -1 * m.wind
m.wf.capacity.x <= 100
m.wf.capacity.x >= 10
m.wf.operate.prep(norm=True) <= [0.9, 0.8, 0.5, 0.7]
m.wf.capacity[m.usd.spend] == 990637 + 3354
m.wf.operate[m.usd.spend] == 49

m.pv = Process()
m.pv(m.power) == -1 * m.solar
m.pv.capacity.x <= 100
m.pv.capacity.x >= 10
m.pv.operate.prep(norm=True) <= [0.6, 0.8, 0.9, 0.7]
m.pv.capacity[m.usd.spend] == 567000 + 872046
m.pv.operate[m.usd.spend] == 90000

m.lii = Storage()
m.lii(m.power) == 0.9
m.lii.capacity.x <= 100
m.lii.capacity.x >= 10
m.lii.capacity[m.usd.spend] == 1302182 + 41432
m.lii.inventory[m.usd.spend] == 2000
m.lii.charge.capacity <= 100
m.lii.charge.operate <= 1
m.lii.discharge.capacity <= 100
m.lii.discharge.operate <= 1

--- General Resource Balance for solar in (l, q): initializing constraint, adding consume(solar, l, q)
    Completed in 0.00013184547424316406 seconds
--- Binding consume in domain (solar, l, q)
    Completed in 7.104873657226562e-05 seconds
--- General Resource Balance for wind in (l, y): initializing constraint, adding consume(wind, l, y)
    Completed in 6.341934204101562e-05 seconds
--- Binding consume in domain (wind, l, y)
    Completed in 3.886222839355469e-05 seconds
--- General Resource Balance for power in (l, q): initializing constraint, adding release(power, l, q)
    Completed in 0.0002281665802001953 seconds
--- Binding release in domain (power, l, q)
    Completed in 0.00012350082397460938 seconds
--- Binding capacity in domain (wf, l, y)
    Completed in 0.00010728836059570312 seconds
--- Binding capacity in domain (wf, l, y)
    Completed in 9.298324584960938e-05 seconds
--- Binding operate in domain (wf, l, q)
    Completed in 0.00013375282287597656 seconds
--- Mappin

## Indicator Stream 

Indicators scale the impact of some decision or flow and project it only a common dimension allowing the calculation of overall impact as a single metric value

In this case, we consider global warming potential (GWP).

In [20]:
m.gwp = Environ()

Now say that we want to consider the impact of setting up operations (specific to the act of construction)

These can be provided as calculations 

In [21]:
m.wf.capacity[m.gwp.emit] == 1000
m.pv.capacity[m.gwp.emit] == 2000
m.lii.capacity[m.gwp.emit] == 3000
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

Note that impact streams are mapped across scales but not balanced, unless a negative (relatively) impact is also assessed! In the case of emission, this would be abatement.

## Material Stream

Note that all the objects found in energia.components.commodity.misc are still resources (Land, Material, etc.)

This will allow for the modeling of an expansive type of processes.

Examples:

1. land clearance to transform agricultural land into industrial, or land remediation 
2. resource intense material recycling 
3. some resource flowing being treated as emission flows 

The possibilities are vast... The advanced modeler may prefer to use Resource() for everything [see Resource Task Network (RTN) Framework]

In the following example, we consider a limit to cement consumption, as also an expense and gwp impact associated with it

In [22]:
m.cement = Material()
m.cement.consume <= 1000000
m.cement.consume[m.usd.spend] == 17
m.cement.consume[m.gwp.emit] == 0.9

--- General Resource Balance for cement in (l, y): initializing constraint, adding consume(cement, l, y)
    Completed in 0.00011682510375976562 seconds
--- Binding consume in domain (cement, l, y)
    Completed in 5.054473876953125e-05 seconds


Next, we provide details of the use of cement across all operations 

In [23]:
m.wf.capacity[m.cement.use] == 400
m.pv.capacity[m.cement.use] == 560
m.lii.capacity[m.cement.use] == 300

--- General Resource Balance for cement in (l, y): adding use(cement, l, y, capacity, wf)
    Completed in 0.00010013580322265625 seconds
--- General Resource Balance for cement in (l, y): adding use(cement, l, y, capacity, pv)
    Completed in 7.43865966796875e-05 seconds
--- General Resource Balance for cement in (l, y): adding use(cement, l, y, invcapacity, power.lii)
    Completed in 6.127357482910156e-05 seconds


Material use is summed up across all operations, and adheres to an upper bound in terms of consumption. 

In [24]:
m.cement.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

GWP impact is now the sum of impact from material use as well as constuction 

In [25]:
m.emit.show()

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Locate Operations

In [26]:
m.network.locate(m.wf, m.pv, m.lii)

--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, wf)
    Completed in 0.00016045570373535156 seconds
--- General Resource Balance for wind in (l, y): adding expend(wind, l, y, operate, wf)
    Completed in 7.176399230957031e-05 seconds
--- General Resource Balance for power in (l, q): adding produce(power, l, q, operate, pv)
    Completed in 9.918212890625e-05 seconds
--- General Resource Balance for solar in (l, q): adding expend(solar, l, q, operate, pv)
    Completed in 8.821487426757812e-05 seconds
--- Assuming  power.lii inventory capacity is unbounded in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, y)
--- Assuming inventory of power.lii is bound by inventory capacity in (l, q)
--- Mapping inventory: (power.lii, l, q) → (power.lii, l, y)
    Completed in 0.000s
--- General Resource Balance for power.lii in (l, q): initializing constraint, adding inventory(power.lii, l, q)
    Completed in 0.00017929077148437

## Optimization 

Let us minimize GWP this time. Note that m.usd.spend.opt() can still be used!

In [27]:
m.gwp.emit.opt()

--- Mapping emit: (gwp, l, y, capacity, wf) → (gwp, l, y)
    Completed in 0.000s
--- Mapping emit: (gwp, l, y, capacity, pv) → (gwp, l, y)
    Completed in 0.000s
--- Mapping emit: (gwp, l, y, invcapacity, power.lii) → (gwp, l, y)
    Completed in 0.000s
--- Mapping emit: (gwp, l, y, consume, cement) → (gwp, l, y)
    Completed in 0.000s
--- Generating Program(example3).mps
--- Creating gurobi model for Program(example3)
Read MPS format model from file Program(example3).mps
Reading time = 0.00 seconds
PROGRAM(EXAMPLE3): 95 rows, 87 columns, 217 nonzeros
--- Optimizing Program(example3) using gurobi
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 95 rows, 87 columns and 217 nonzeros
Model fingerprint: 0x39a95d48
Variable types: 84 continuous, 3 integer (3 binary)
Coefficie

In [28]:
len(m.variables)

87

In [29]:
m.show()

# Mathematical Program for Program(example3)

<br><br>

## Index Sets

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

###  Function Sets

<IPython.core.display.Math object>

In [30]:
m.sol(False)

# Solution for Program(example3)

<br><br>

## Objective

<IPython.core.display.Math object>

<br><br>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>